# Daily News Summarizer

This notebook implements a simple news summarization app that:
1. Asks for a topic of interest
2. Searches for recent news using SerpAPI
3. Summarizes the findings using LiteLLM
4. Presents the top 3 important items to know

## 1. Install Required Dependencies

In [38]:
# Install required packages
%pip install google-search-results litellm requests beautifulsoup4

5972.75s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. Import Libraries

In [39]:
import os
import time
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
from serpapi import GoogleSearch
import litellm

## 3. Configure API Keys

In [40]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables
SERPAPI_KEY = os.getenv('SERP_AI_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not SERPAPI_KEY or not OPENAI_API_KEY:
    raise ValueError("API keys not found in .env file. Please check your .env file configuration.")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## 4. Get User Input

In [41]:
def get_topic():
    """Ask the user for a news topic to search for"""
    topic = input("Enter a topic to get news about: ")
    return topic

## 5. Search for News

In [42]:
def search_news(topic):
    """Search for news articles on the given topic using SerpAPI"""
    print(f"Searching for news about '{topic}'...")
    
    # Calculate date for last 24 hours
    yesterday = datetime.now() - timedelta(days=1)
    date_str = yesterday.strftime("%Y-%m-%d")
    
    # Set up the search parameters
    params = {
        "engine": "google_news",
        "q": topic,
        "tbm": "nws",
        "tbs": f"cdr:1,cd_min:{date_str},cd_max:today",
        "num": 10,  # Get top 10 results
        "api_key": SERPAPI_KEY
    }
    
    # Execute the search
    search = GoogleSearch(params)
    results = search.get_dict()
    
    # Check if we have news results
    if "news_results" not in results:
        print("No news found for this topic in the last 24 hours.")
        return []
    
    return results["news_results"]

## 6. Extract Article Content

In [43]:
def extract_article_content(url):
    """Extract the main text content from a news article URL"""
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style", "nav", "footer", "header"]):
            script.extract()
        
        # Get text
        text = soup.get_text(separator=' ', strip=True)
        
        # Clean up text (remove extra whitespace)
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        # Limit text length for API constraints
        return text[:4000]  # Limit to 4000 chars to avoid token limits
    except Exception as e:
        print(f"Error extracting content from {url}: {e}")
        return ""

## 7. Summarize News with LiteLLM

In [44]:
def summarize_news(articles):
    """Summarize the news articles using LiteLLM"""
    if not articles:
        return "No news articles found to summarize."
    
    print("Collecting and summarizing news articles...")
    
    # Prepare context for the LLM
    article_texts = []
    for i, article in enumerate(articles[:5]):  # Process top 5 articles
        title = article.get("title", "No title")
        source = article.get("source", "Unknown source")
        link = article.get("link", "")
        snippet = article.get("snippet", "")
        
        print(f"Processing article {i+1}: {title}")
        
        # Get full content if possible
        content = extract_article_content(link)
        if not content:
            content = snippet  # Fall back to snippet if extraction fails
            
        article_texts.append(f"ARTICLE {i+1}:\nTitle: {title}\nSource: {source}\nContent: {content}\n")
    
    # Combine all articles
    all_articles = "\n\n".join(article_texts)
    
    # Create prompt for the LLM
    prompt = f"""Based on the following news articles, provide a concise summary highlighting the 3 most important things someone should know about this topic and why they matter. Format your response as a numbered list (1, 2, 3) with a brief explanation for each point.\n\n{all_articles}"""
    
    # Call LiteLLM to summarize
    try:
        response = litellm.completion(
            model="gpt-4o",  # You can change this to another model
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=1000
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in LLM summarization: {e}")
        return "Error generating summary. Please check your API keys and try again."

## 8. Main Function

In [45]:
def get_daily_news():
    """Main function to run the news summarizer"""
    # Get topic from user
    topic = get_topic()
    
    # Search for news
    articles = search_news(topic)
    
    if not articles:
        return "No news found for this topic in the last 24 hours."
    
    # Summarize the news
    summary = summarize_news(articles)
    
    print("\n" + "="*50)
    print(f"TOP NEWS SUMMARY FOR: {topic.upper()}")
    print("="*50)
    print(summary)
    print("="*50)
    
    return summary

## 9. Run the News Summarizer


In [46]:
# Run the news summarizer
get_daily_news()

Searching for news about 'India vs Pakistan'...
Processing article 1: News about ceasefire • India • Pakistan
Error extracting content from : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Processing article 2: In-depth
Error extracting content from : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Processing article 3: For context
Error extracting content from : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Processing article 4: India-Pakistan truce appears to hold despite accusations of violations
Processing article 5: At least two Indian jets appear to have crashed during Pakistan strikes, visuals show
Error extracting content from https://www.washingtonpost.com/world/2025/05/09/fighter-jets-india-pakistan-attack/: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=10)

TOP NEWS SUMMARY FOR: INDIA VS PAKISTAN
Based on the provided articles, here are the three most important points regarding t

'Based on the provided articles, here are the three most important points regarding the India-Pakistan ceasefire:\n\n1. **Ceasefire Holding Amid Accusations**: Despite initial accusations of violations, the ceasefire between India and Pakistan appears to be holding. This is significant as it suggests a temporary de-escalation of tensions in the region, which has historically been a flashpoint for conflict between the two nuclear-armed neighbors. The stability of this truce is crucial for regional peace and security.\n\n2. **US Mediation Role**: The ceasefire was reportedly facilitated by overnight diplomacy involving the United States, with President Trump claiming a role in securing the agreement. This highlights the influence and involvement of international actors in regional conflicts, emphasizing the global implications of the India-Pakistan tensions and the importance of diplomatic interventions in conflict resolution.\n\n3. **Underlying Tensions and Skepticism**: The ceasefire f